<a href="https://colab.research.google.com/github/Devesh-Patodia/NLP-J038/blob/home-assignments/NMF_on_Enron_emails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import nltk
import re

In [0]:
df = pd.read_csv('/content/drive/My Drive/emails.csv')
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [0]:
df.isna().sum()

file       0
message    0
dtype: int64

In [0]:
df.shape

(517401, 2)

In [0]:
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()

from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
def pipeline(text):
  text=re.sub('[^a-zA-Z]', " ", str(text))
  text=text.lower()
  stop_words = set(stopwords.words('english'))
  word_tokens = word_tokenize(text)
  filtered_sentence = [w for w in word_tokens if not w in stop_words]
  filtered_sentence = nltk.pos_tag(text.split())
  text = lem.lemmatize(text)
  filtered_sentence = [lem.lemmatize(word) for word in word_tokens]
  return text

In [0]:
data = df[:100]

data['file_pipeline'] = data['file'].apply(pipeline)
data['message_pipeline'] = data['message'].apply(pipeline)
data.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,file,message,file_pipeline,message_pipeline
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,allen p sent mail,message id javamail e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,allen p sent mail,message id javamail e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,allen p sent mail,message id javamail e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,allen p sent mail,message id javamail e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,allen p sent mail,message id javamail e...


In [0]:
model = vect.fit_transform(data['message_pipeline'])

In [0]:
from sklearn.decomposition import NMF

In [0]:
model = pd.DataFrame(model.toarray(), columns=vect.get_feature_names())

In [0]:
nmf_model = NMF(n_components=4, init='nndsvd', random_state=0)

In [0]:
W = nmf_model.fit_transform(model)
print(W)

[[1.98063054e-01 0.00000000e+00 0.00000000e+00 4.04343970e-02]
 [9.48675825e-02 2.13107127e-01 0.00000000e+00 0.00000000e+00]
 [2.12779737e-01 0.00000000e+00 0.00000000e+00 9.41622637e-04]
 [1.90882698e-01 1.15000251e-01 0.00000000e+00 0.00000000e+00]
 [2.66145878e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.46473151e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.52764378e-01 0.00000000e+00 1.93421301e-02 4.03086858e-04]
 [1.99295551e-01 0.00000000e+00 0.00000000e+00 3.99996354e-02]
 [1.19894509e-01 0.00000000e+00 1.49037724e-02 1.88796562e-01]
 [6.63148698e-02 1.47807463e-01 3.49177104e-02 1.09957728e-02]
 [1.53659820e-01 4.06690191e-02 5.99782512e-03 8.99429804e-03]
 [1.37495644e-01 1.05196732e-01 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 3.80393668e-01 1.46978510e-01 0.00000000e+00]
 [0.00000000e+00 3.80393668e-01 1.46978510e-01 0.00000000e+00]
 [1.95358556e-01 6.08513777e-02 0.00000000e+00 3.37381221e-02]
 [2.10931875e-01 0.00000000e+00 0.00000000e+00 1.139709

In [0]:
H = nmf_model.components_
print(H)

[[0.         0.         0.06385289 ... 0.         0.         0.        ]
 [0.04765654 0.02135081 0.03185547 ... 0.01307655 0.00164285 0.00779706]
 [0.01116985 0.00025773 0.         ... 0.         0.00144198 0.        ]
 [0.         0.04215202 0.         ... 0.         0.         0.        ]]


In [0]:
def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(documents[doc_index])

In [0]:
from sklearn.decomposition import LatentDirichletAllocation

In [0]:
lda = LatentDirichletAllocation(n_components=5, random_state=0)

In [0]:
lda.fit(model)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [0]:
lda.components_

array([[0.20001443, 0.20001359, 0.2000111 , ..., 0.20001532, 0.20001014,
        0.20001887],
       [0.20000946, 0.20001396, 0.20064348, ..., 0.20001575, 0.20001043,
        0.20001941],
       [0.68284313, 0.59559819, 1.45525302, ..., 0.27906188, 0.21877037,
        0.27549164],
       [0.20000956, 0.20001411, 0.20041779, ..., 0.20001592, 0.20001042,
        0.20001961],
       [0.20000965, 0.20001424, 0.20001163, ..., 0.20001607, 0.20001052,
        0.20001979]])